<a href="https://colab.research.google.com/github/IyadSultan/Basic_Flask_site/blob/master/Pathology_extraction/Extract_path_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai

In [3]:
import os
# import openai

from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from typing import Optional
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
import pandas as pd
from tqdm import tqdm
# from dotenv import load_dotenv, find_dotenv
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')
from pydantic import BaseModel, Field
model = ChatOpenAI(temperature=0,model_name="gpt-4-0125-preview")

import json
import pandas as pd
from typing import Dict, Any, List

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [35]:
df = pd.read_csv("/content/pathology_reports.csv", encoding='MacRoman')

df=df[df["include"]==1]
df = df.head(4)
df.shape



(4, 4)

In [41]:


# First, create an empty JSON file
with open('pathology_report_data.json', 'w') as file:
    json.dump({}, file)

# Define the Information class
class Information(BaseModel):
    """Information to extract."""
    #Specimen Information:
    malignancy: str = Field(description="Is this report for a malignancy or a malignant condition? answer using Yes or No")
    Primary_site: str = Field(description="Site of primary tumor if known (e.g. Breast, or Unknown primary)")
    Specimen_type: str = Field(description="Specimen type (e.g., biopsy, mastectomy, lumpectomy)")
    Specimen_site: str = Field(description="Specimen site (left/right breast, specific quadrant)")
    #Macroscopic Description:
    specimen_size: str = Field(description="Size of the specimen n*n*n and/or in ml")
    specimen_weight: float = Field(description=" weight of the specimen in grams (e.g. 50 for a tumor that is 50 grams)")
    #Microscopic Description:
    Histologic_type: str = Field(description="Histologic type of breast cancer , try to use ICD-O classification (e.g., invasive ductal carcinoma, invasive lobular carcinoma)")
    tumor_grade : str = Field(description="Grade of the tumor (e.g., Nottingham grade)")
    tumor_size : str = Field(description="Tumor size")
    tumor_margins : str = Field(description="Description of the tumor margins (involved or uninvolved)")
    #Tumor Staging:
    TNM_staging : str = Field(description="TNM staging (Tumor size, Node status, Metastasis)")
    #Lymph Node Status:
    Number_of_lymph_nodes_examined  : str = Field(description="Number of lymph nodes examined, 0 if none")
    Number_of_lymph_nodes_involved  : str = Field(description="Number of lymph nodes involved, 0 if none")
    Location_of_involved_lymph_nodes  : str = Field(description="Location of involved lymph nodes")
    #Additional Findings:
    Presence_of_lymphovascular_invasion  : str = Field(description="Presence of lymphovascular invasion")
    Presence_of_ductal_carcinoma_in_situ : str = Field(description="Presence of ductal carcinoma in situ (DCIS)")
    Necrosis: str = Field(description="Percentage of necrosis if present")
    # Treatment Recommendations (if included):
    # Suggested_therapeutic  : str = Field(description="Suggested therapeutic options based on pathology findings")
    # immunohistochemistry
    Immunohistochemsitry: str = Field(description="""
    Any immunohistochemistry stains done with their results, e.g. ER, PR, HER2, Ki67;
    results presented as Positive, Negative, score or number with no other text;
    Instead of saying 'Estrogen ReceptorPositive, strong, in 95% of tumor cell nuclei' just say 'Estrogen Receptor, Positive'
    results should be seperated using |""")


convert_pydantic_to_openai_function(Information)
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})



prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are an expert pathologist working at King Hussein Cancer Center.
    Extract relevant information from the provided request.
    Extract ONLY the information presented in the initial request; don't add anything else.
    Return partial information if something is missing.
    If you have a tumor and lymph nodes in the sample, extract information about the tumor(e.g. tumor size, site, pathology) .
    """),
    ("human", "{input}")
])


extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()


def append_to_json(file_path, data):
    with open(file_path, 'r+') as file:
        file_data = json.load(file)
        if not isinstance(file_data, list):
            file_data = []
        file_data.append(data)
        file.seek(0)
        json.dump(file_data, file, indent=4)

def process_dataframe_and_update_json(df: pd.DataFrame, json_file_path: str) -> None:
    for _, row in df.iterrows():
        result=extraction_chain.invoke({"input":row['pathalogy_note']})
        pathology_data = {
            "krn": row['krn'],
            "kate": row['kate'],
            "pathology_note": row['pathalogy_note'],
            "extracted_data": result
        }
        append_to_json(json_file_path, pathology_data)



# Note: This function stub assumes you have a mechanism to extract the required fields from the pathology_report column.
# You would need to integrate the actual extraction logic based on your model or extraction method.


In [42]:
# Assuming df is your dataframe with columns: krn, kate, and pathology_report
process_dataframe_and_update_json(df, 'pathology_report_data.json')

In [43]:
# prompt: import pathology_report_date.json and print it


with open('pathology_report_data.json', 'r') as file:
    result = json.load(file)

for report in result:
    print(f"krn: {report['krn']}, kate: {report['kate']}")
    for attribute, value in report['extracted_data'].items():
        print(f"{attribute}: {value}")
    print("-" * 50)  # Separator for each report



krn: 124120, kate: 7/12/2017
malignancy: No
Primary_site: Thyroid
Specimen_type: FNA
Specimen_site: Right thyroid lobe, Left thyroid lobe, Isthmus
specimen_size: Not provided
specimen_weight: 0
Histologic_type: Atypical cells of undetermined significance
tumor_grade: Not applicable
tumor_size: Not applicable
tumor_margins: Not applicable
TNM_staging: Not applicable
Number_of_lymph_nodes_examined: 0
Number_of_lymph_nodes_involved: 0
Location_of_involved_lymph_nodes: Not applicable
Presence_of_lymphovascular_invasion: Not applicable
Presence_of_ductal_carcinoma_in_situ: Not applicable
Necrosis: Not applicable
Immunohistochemsitry: Not provided
--------------------------------------------------
krn: 176865, kate: 10/9/2019
malignancy: Yes
Primary_site: Right breast
Specimen_type: Tru cut biopsy
Specimen_site: Right breast
specimen_size: Unknown
specimen_weight: 0
Histologic_type: Invasive carcinoma with lobular features
tumor_grade: 1
tumor_size: Tiny fragments
tumor_margins: Unknown
TNM_